In [1]:
import numpy as np
import math
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl

frames_dir = "test_frames_5_2/"

#camera information based on the Kinect v2 hardware
CameraParams = {
  "cx":254.878,
  "cy":205.395,
  "fx":365.456,
  "fy":365.456,
  "k1":0.0905474,
  "k2":-0.26819,
  "k3":0.0950862,
  "p1":0.0,
  "p2":0.0,
}

# Kinect's physical orientation in the real world.
CameraPosition = {
    "x": -3, # actual position in meters of kinect sensor relative to the viewport's center.
    "y": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "z": 0.5, # height in meters of actual kinect sensor from the floor.
    "roll": 0, # angle in degrees of sensor's roll (used for INU input - trig function for this is commented out by default).
    "azimuth": 0, # sensor's yaw angle in degrees.
    "elevation": -16, # sensor's pitch angle in degrees.
}

def depthMatrixToPointCloudPos(z, scale=1000):
    #bacically this is a vectorized version of depthToPointCloudPos()
    C, R = np.indices(z.shape)

    R = np.subtract(R, CameraParams['cx'])
    R = np.multiply(R, z)
    R = np.divide(R, CameraParams['fx'] * scale)

    C = np.subtract(C, CameraParams['cy'])
    C = np.multiply(C, z)
    C = np.divide(C, CameraParams['fy'] * scale)

    return np.column_stack((z.ravel() / scale, R.ravel(), -C.ravel()))

def depthToPointCloudPos(x_d, y_d, z, scale=1000):
    # This runs in Python slowly as it is required to be called from within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # calculate the real-world xyz vertex coordinate from the raw depth data (one vertex at a time).
    x = (x_d - CameraParams['cx']) * z / CameraParams['fx']
    y = (y_d - CameraParams['cy']) * z / CameraParams['fy']

    return x / scale, y / scale, z / scale

def applyCameraMatrixOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # bacically this is a vectorized version of applyCameraOrientation()
    # uses same trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[:, ax1] ** 2 + pt[:, ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[:, ax2], pt[:, ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[:, ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[:, ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(1, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(0, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

In [2]:


def process_frame(i):
    focal_x = 70.6
    depth_frame = np.load(frames_dir + str(i) + ".npy")
    obstacles = np.zeros(depth_frame.shape)
    img = depth_frame / 4500.
    imgray = np.uint8(img * 255)
    sure_bg_dilation = cv2.getTrackbarPos('bg_dilate', 'depth')
    sure_bg_erosion = cv2.getTrackbarPos('bg_erode', 'depth')
    thresh_dilation = cv2.getTrackbarPos('thresh_dilate', 'depth')
    thresh_erosion = cv2.getTrackbarPos('thresh_erode', 'depth')
    opening_iter = cv2.getTrackbarPos('opening', 'depth')
    unknown_erosion = cv2.getTrackbarPos('unknown_erode', 'depth')
    kernel_size = cv2.getTrackbarPos('kernel_size', 'depth')
    ret,thresh = cv2.threshold(imgray,0,255,cv2.THRESH_BINARY)

    #noise removal
    kernel = np.ones((kernel_size,kernel_size),np.uint8)
    thresh = cv2.erode(thresh, kernel, iterations = thresh_erosion)
    thresh = cv2.dilate(thresh, kernel, iterations = thresh_dilation)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = opening_iter)
    #gradient calculation
    gradient = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)
    dilation = cv2.dilate(thresh,kernel,iterations = 1)
    #sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=sure_bg_dilation)
    sure_bg = cv2.erode(sure_bg, kernel, iterations = sure_bg_erosion)
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    #finding unknown region
    #unknown = cv2.subtract(gradient,sure_bg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    #unknown = cv2.subtract(unknown,gradient)
    #unknown = cv2.medianBlur(unknown,5)
    unknown = cv2.erode(unknown, kernel, iterations = unknown_erosion)
    color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(unknown,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent cirlce
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)

            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 20
            HIGH_DIAMETER_BOUND = 150

            HIGH_DISTANCE_BOUND = 4500
            #Original tolerances were 20 and 150

            if(equi_diameter>LOW_DIAMETER_BOUND and equi_diameter<HIGH_DIAMETER_BOUND): #range needs to be tweaked
                mask = np.zeros_like(imgray)
                ellipse = cv2.fitEllipse(cntr)
                x,y,obj_length,obj_height = cv2.boundingRect(cntr)
                rect = cv2.minAreaRect(cntr)
                
                equi_diameter = obj_length
                
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1)
                rows,cols = mask.shape
                #shift mask down to match obstacle, not edge
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]])
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3)
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask)
                img_fg = cv2.medianBlur(img_fg,5)
                print (img_fg.shape)
                print (obstacles.shape)
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles))



                # Experimenting with different blur settings
                #img_fg = cv2.GaussianBlur(img_fg, (5,5), 0)

                #mean_val = cv2.mean(img_fg)[0] #returns mean value of each channel, we only want first channel
                non_zero_mean = np.median(img_fg[img_fg.nonzero()])
                mean_val = non_zero_mean
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)

                moment = cv2.moments(cntr)
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND:
                    coords = depthToPointCloudPos(cx, cy, mean_val)
                    
                    mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val

                    img = cv2.ellipse(color,ellipse,(0,255,0),2)
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX

                    cv2.putText(img, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(img, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to fit ellipse")

    
    cv2.imshow("unknown", unknown)
    cv2.imshow("obstacles", obstacles)
    cv2.imshow("depth", img)
    key = cv2.waitKey(delay=30)
    #time.sleep(0.1)
    
    return obstacles

def plot_cloud(i, sp2, obstacles):
    depth_frame = np.load(frames_dir + str(i) + ".npy")
    img = depth_frame / 4500.
    xyz_arr = depthMatrixToPointCloudPos(depth_frame)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    obstacles_arr = depthMatrixToPointCloudPos(obstacles)
    obstacles_arr = applyCameraMatrixOrientation(obstacles_arr)
    colors = ((1.0, 1.0, 1.0, 1.0))
    colors = np.uint8(obstacles_arr)
    #colors = np.divide(colors, 255)
    #colors = colors.reshape(colors.shape[0] * colors.shape[1], 4 )
    colors = colors[:, :3:] #BGRA to BGR (slices out the alpha channel)  
    colors = colors[...,::-1] #BGR to RGB
    # Calculate a dynamic vertex size based on window dimensions and camera's position - To become the "size" input for the scatterplot's setData() function.
    v_rate = 5.0 # Rate that vertex sizes will increase as zoom level increases (adjust this to any desired value).
    v_scale = np.float32(v_rate) / gl_widget.opts['distance'] # Vertex size increases as the camera is "zoomed" towards center of view.
    v_offset = (gl_widget.geometry().width() / 1000)**2 # Vertex size is offset based on actual width of the viewport.
    v_size = v_scale + v_offset
    #cloud = PyntCloud(points)
    #cloud.plot(point_size=0.05, opacity=0.6)

    # Show the data in a scatter plot
    sp2.setData(pos=xyz_arr, color=colors, size=v_size)


In [ ]:
def update():
    for i in range(100,229):
        obstacles = process_frame(i)
        plot_cloud(i,sp2,obstacles)
def update_image(i):
    plot_cloud(175,sp2,process_frame(175))
        
cv2.namedWindow('depth')
cv2.createTrackbar('bg_dilate', 'depth', 0, 10, update_image)
cv2.createTrackbar('bg_erode', 'depth', 0, 10, update_image)
cv2.createTrackbar('thresh_dilate', 'depth', 0, 10, update_image)
cv2.createTrackbar('thresh_erode', 'depth', 0, 10, update_image)
cv2.createTrackbar('opening', 'depth', 0, 10, update_image)
cv2.createTrackbar('unknown_erode', 'depth', 0, 10, update_image)
cv2.createTrackbar('kernel_size', 'depth', 0, 10, update_image)

#QT app
app = QtGui.QApplication([])
gl_widget = gl.GLViewWidget()
gl_widget.show()
gl_grid = gl.GLGridItem()
gl_widget.addItem(gl_grid)
#initialize some points data
pos = np.zeros((1,3))


sp2 = gl.GLScatterPlotItem(pos=pos)
sp2.setGLOptions('opaque') # Ensures not to allow vertexes located behinde other vertexes to be seen.

gl_widget.addItem(sp2)

t = QtCore.QTimer()
#t.timeout.connect(update)
t.start(50)

process_frame(175)
    
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

sys.exit(0)


(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)

C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\jumpr\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in less


(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)

(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)
(424, 512)


In [ ]:
depth_frame = np.load("test_frames/0.npy")
img = depth_frame / 4500.
xyz_arr = depthMatrixToPointCloudPos(depth_frame)
print (xyz_arr)
points = pd.DataFrame(xyz_arr, columns=['z', 'x', 'y'])
print (points)
colors = (np.random.uniform(size=(217088, 3)) * 255).astype(np.uint8)
points[['red', 'blue', 'green']] = pd.DataFrame(colors, index=points.index)
#cloud = PyntCloud(points)
#cloud.plot(point_size=0.05, opacity=0.6)